## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

###  scrape the Wikipedia page and transform the data into a pandas dataframe

In [19]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [4]:
data = []
columns = []
My_table = soup.find('table',{'class':'wikitable'})
for index, tr in enumerate(My_table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
#First row as header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#converting to Pandas DataFrame
df = pd.DataFrame(data = data,columns = columns)
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [5]:
My_table = soup.find('table',{'class':'wikitable'})

In [6]:
# Ignore cells with a borough that is Not assigned.
df = df[df['Borough'] != 'Not assigned']

#update index to be postcode
if(df.index.name != 'Postal code'):
    df = df.set_index('Postal code')
df.head()

,Borough,Neighborhood
Postal code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [7]:
#More than one neighborhood can exist in one postal code area. 
#For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
#These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

df["Neighborhood"] = df.groupby("Postal code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))


In [8]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df['Neighborhood'].replace('Not assigned', df['Borough'],inplace=True)
df.head()

,Borough,Neighborhood
Postal code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [15]:
#drop duplicates
df = df.drop_duplicates()
 
#final dataframe head   
df.head(10)

,Borough,Neighborhood
Postal code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,Malvern / Rouge
M3B,North York,Don Mills
M4B,East York,Parkview Hill / Woodbine Gardens


In [10]:
#used the .shape method to print the number of rows of dataframe.
df.shape

(98, 2)

In [14]:
!pip install geocoder

In [18]:
df['Latitude'], df['Longitude'] = zip(*post_df['Postcode'].apply(get_geocoder))

NameError: name 'post_df' is not defined